## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import get_all_vars
import yaml
import warnings
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings

bucket = S3BucketUtils()

warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

with open(r'./parameters/for_properly_used_inquiries_vars.yaml') as file:
    for_properly_used_inquiries_vars = yaml.load(file, Loader=yaml.FullLoader)

def get_key_based_on_value_in_a_dict(dict_, value_of_interest):
    for key, value in dict_.items():
        if value == value_of_interest:
            return key

def get_model_number(var_, model_names):
    if len(model_names[model_names['model_name']==var_]['model_number'])>0:
        model_number = model_names[model_names['model_name']==var_]['model_number'].iloc[0]
    else:
        multiple_vars = model_names[(model_names['model_name'].apply(lambda x: ',' in x))]
        for i in range(0, len(multiple_vars)):
            if var_ in multiple_vars['model_name'].iloc[i]:
                model_number = multiple_vars['model_number'].iloc[i]
                break
    return model_number

def get_pairs_of_variables(churn_based_on_behaviour_dir, date_dir, model_type):
    vars_periods_to_look_at_thresholds_to_use = \
    bucket.load_csv_from_s3(file_name = churn_based_on_behaviour_dir + 'data/' + date_dir +\
                     '/vars_periods_to_look_at_thresholds_to_use.csv')

    period_to_look_at =\
    vars_periods_to_look_at_thresholds_to_use[['variable_base_name', 'period_to_look_at', 'number_of_months']].\
    apply(lambda x: (x['variable_base_name'], x['period_to_look_at']), axis=1).tolist()


    (all_vars, all_vars_base_names) = get_all_vars.main()
    first_var = dict.fromkeys(all_vars_base_names.values())
    second_var = dict.fromkeys(all_vars_base_names.values())

    if model_type == 'started_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]
            second_var[period_to_look_at[i][0]] = \
            'had_'+period_to_look_at[i][0]+'_before_and_didnt_'+period_to_look_at[i][1]

        for key in first_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                first_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_'+period_to_look_at_tmp
        for key in second_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                second_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_before_and_didnt_'+period_to_look_at_tmp

    elif model_type == 'stopped_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_before'
            second_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]

        for key in first_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                first_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_before'
        for key in second_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                second_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_'+period_to_look_at_tmp


    return (first_var, second_var)

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

var_of_interest = input("Variable we're interested in: ")
# had_private_parties_submissions_last_2_months


without_properly_used_vars = input('Without properly used inquiries variables (Yes/No)? ')
if without_properly_used_vars == 'Yes':
    without_properly_used_vars = True
    combinations_of_vars_dir = 'combinations_of_variables_that_are_not_dependent/combinations_and_names_without_properly_used_inquiries_vars'
elif without_properly_used_vars == 'No':
    without_properly_used_vars = False
    combinations_of_vars_dir = 'combinations_of_variables_that_are_not_dependent'
    
    
(first_var, second_var) = \
get_pairs_of_variables(churn_based_on_behaviour_dir='churn_analysis_based_on_behaviour/', date_dir=date_dir,\
                       model_type=model_type)

if var_of_interest in first_var.values():
    base_var_ = get_key_based_on_value_in_a_dict(dict_=first_var, value_of_interest=var_of_interest)
elif var_of_interest in second_var.values():
    base_var_ = get_key_based_on_value_in_a_dict(dict_=second_var, value_of_interest=var_of_interest)
else:
    base_var_ = var_of_interest
    
if 'properly' in var_of_interest:
    base_var_ = for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][base_var_]
    
model_names = \
bucket.load_csv_from_s3(file_name='churn_analysis_based_on_behaviour/'+combinations_of_vars_dir+'/model_names.csv')
model_number = get_model_number(var_=base_var_, model_names=model_names)

Variable we're interested in: had_catering_submissions_last_3_months
Without properly used inquiries variables (Yes/No)? Yes


In [4]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']
for data_set_name in ['ALL_spots_with_CB_cancellation_confirmed', 'ALL_spots_with_CB_cancellation_requested',\
                     'ALL_spots_wo_CB_cancellation_confirmed', 'ALL_spots_wo_CB_cancellation_requested',\
                     'CAN_CANCEL_spots_with_CB_cancellation_confirmed', 'CAN_CANCEL_spots_with_CB_cancellation_requested',\
                     'CAN_CANCEL_spots_wo_CB_cancellation_confirmed', 'CAN_CANCEL_spots_wo_CB_cancellation_requested']:
    with open(r'./parameters/data_sets.yaml') as file:
        data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)

    spots_set = data_sets_parameters[data_set_name]['spots_set']
    with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
    event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

    if with_wo_CB == 'with_CB':
        with_wo_CB_boolean = True
    else:
        with_wo_CB_boolean = False

    if event_date_full_name == 'cancellation_requested':
        event_date = 'canc_req'
    elif event_date_full_name == 'cancellation_confirmed':
        event_date = 'canc_conf'
        
    ##### Model 0: all variables - prepare data for the model ####
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=0, dir_name=combinations_of_vars_dir+'/')
    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = base_cols + variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars


    #### get data for the model ###
    (data, base_df, df_timeline_all_vars) = \
    prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                                with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                                C = 100)
    
    
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name=combinations_of_vars_dir+'/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name=combinations_of_vars_dir+'/')
    
     #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    
    (ctv, all_summaries, conditions_described) = \
    fit_tv_cox_models.fit_the_models_and_return_all_intermediate_results(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    for condition in all_summaries.keys():
#         print(condition)
#         print(len(all_summaries[condition]))
        if without_properly_used_vars == True:
            fit_tv_cox_models.save_results_for_a_specific_variable(df = all_summaries[condition].reset_index(),\
                                            condition=condition,\
                                            var_of_interest=var_of_interest,\
                                            date_of_analysis=date_of_analysis,\
                                            data_dir=churn_based_on_behaviour_dir,\
                                            dir_name='exports/coefficients_and_pvalues',\
                                            results_name='without_properly_used_vars_coef_and_pvalues',\
                                            spots_set=spots_set,\
                                            with_wo_CB=with_wo_CB,\
                                            event_date_type=event_date_full_name,\
                                            model_type=model_type)
        else:
            fit_tv_cox_models.save_results_for_a_specific_variable(df = all_summaries[condition].reset_index(),\
                                            condition=condition,\
                                            var_of_interest=var_of_interest,\
                                            date_of_analysis=date_of_analysis,\
                                            data_dir=churn_based_on_behaviour_dir,\
                                            dir_name='exports/coefficients_and_pvalues',\
                                            results_name='with_properly_used_vars_coef_and_pvalues',\
                                            spots_set=spots_set,\
                                            with_wo_CB=with_wo_CB,\
                                            event_date_type=event_date_full_name,\
                                            model_type=model_type)